## Data extraction and feature preparation for prediction use cases 2 and 3

In [ ]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import plotly 
import plotly.graph_objects as go
from datetime import timedelta

sys.path.insert(0, "./../../")
from src.data_loader import load_game_data, load_game_data_zan
from src.preprocessing.feature_preparation import prepare_player_df, prepare_features_for_season_based_predictions

In [ ]:
%%time
# data_t = pd.read_pickle("../../data/processed/load_data_games_arr_v1.pkl")
games_df = load_game_data_zan(force_recompute=False)
players_df = prepare_player_df()
# display(games_df)
display(players_df)

In [ ]:
# this was moved to load_game_data
if 2 != 3:
    display((games_df["visitor_record_wins"] + games_df["visitor_record_losses"]).max())
    games_df["games_already_played_in_season"] = games_df["visitor_record_wins"] + games_df["visitor_record_losses"]
    all_clubs = set(games_df["visitor_team_id"]).union(games_df["home_team_id"])

    for club_id in all_clubs:
        print(all_clubs, len(all_clubs))
        break
    # games_df[games_df["GAME_ID"]== 2]

## Building the data structure

In [ ]:
games_df["season_name"] = games_df["season_name"].str.replace("-", "-20")

In [ ]:
## buidling json structure
general_nba_info_dict = {
    "all_seasons_info": {
        "team_ids_arr": set(games_df["visitor_team_id"]).union(games_df["home_team_id"]),
        "player_ids_arr": "TODO",
        "games_played": len(games_df.index),
    },
    "seasons": dict.fromkeys(list(games_df["season_name"].unique()), {"games": None, "player_info": None}),
}

# fill dictionary with seasons 
# general_nba_info_dict


for season_name in games_df["season_name"].unique():
    general_nba_info_dict["seasons"][str(season_name)]["games"] = games_df[games_df["season_name"] == season_name]
    general_nba_info_dict["seasons"][str(season_name)]["player_info"] = 0
    

In [ ]:
%%time

display(general_nba_info_dict["seasons"]['2000-2001'])

**X Features**:
- new_players_in_club_count, 
- avg_team (age, weight, height), 
- num_of_games_won or lost before this game,
- last 10 games played(win/loss), 
- sum of true percentage per team, 
- cumulative of (points scored, assists, rebounds etc), 

**Y outcome**
- Use case 1.: 0 for loss, 1 for win
- Use case 2.: same as 1 but with predicted points also

In [ ]:
# all the knowledge moved into this function
prepared_dict = prepare_features_for_season_based_predictions(save_to_file=True)
display(prepared_dict)